In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model = "I:/Llama-2-13b-hf"     # Llama-2-13b-hf
adapter_id = "I:/LlamaCare"          # ruta del LoRA

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # si GPU no soporta bfloat16, usar torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_id)
# model.eval()

# Asegurar pad_token para Llama:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import utils_llm as ul
from tqdm import tqdm
import pandas as pd
import random

subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = random.sample(notes, 2)

input_text = notes[0][4] #"La paciente reporta mejoría moderada con efectos secundarios leves."
instruction = f"""Step-bystep: 
1. Analyse the text, 
2. Select whether the Employment status is explicitly stated, and 
3. Select one of the following labels [employed, unemployed, underemployed, disability, retired, student, unknown]."""
prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
out = model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device),
                     max_new_tokens=200, temperature=0.8)
print(tokenizer.decode(out[0], skip_special_tokens=True))

# OLD CODE

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model = "I:/Llama-2-13b-hf"          # o Llama-2-7b-chat-hf
adapter_id = "I:/LlamaCare"             # LoRA de LlamaCare

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

tok = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             quantization_config=bnb_config,
                                             device_map="auto")
model = PeftModel.from_pretrained(model, adapter_id)  # aplica el LoRA

def chat(prompt, max_new_tokens=256):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tok.decode(out[0], skip_special_tokens=True)

print(chat("Eres un asistente médico. Explica el manejo inicial de DM2 en una simple frase."))


#### Tokenizer and chat_template

In [ ]:
messages = [
    {"role": 'system', "content": 'You are a expert NLP on health datasets.'}, 
    {"role": 'user', "content": "explain what is diabetes type 2."}
]

prompt = tokenizer.apply_chat_template(
    messages, 
    tokenizer=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
out = model.generate(**inputs, max_new_tokens=400, temperature=0, top_p=1)
print(tokenizer.decode(out[0], skip_special_tokens=True))

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
llama2_tpl = """{% for m in messages -%}
{% if m['role'] == 'system' -%}
<s>[INST] <<SYS>>
{{ m['content'] }}
<</SYS>>
{% elif m['role'] == 'user' -%}
{{ m['content'] }} [/INST]
{% elif m['role'] == 'assistant' -%}
{{ m['content'] }}</s>
{% endif -%}
{% endfor -%}
"""

messages = [
  {"role":"system","content":"Your are a NLP expert assistant."},
  {"role":"user","content":f"Analyse the text and determine whether the Employment status is explicitly stated. The text: {input_text}"}
]

prompt = tokenizer.apply_chat_template(messages, chat_template=llama2_tpl,
                                 tokenize=False, add_generation_prompt=True)
out = model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device),
                     max_new_tokens=256, temperature=0.9, top_p=0.9,
                     eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(out[0], skip_special_tokens=True))


[INST] <<SYS>>
Your are a NLP expert assistant.
<</SYS>>
Analyse the text and determine whether the Employment status is explicitly stated. The text: SW consult received via page to address family coping issues.
   SW met w/pt
s mother in the pt
s room. At first pt
s mother asked if we
   needed pt
s organs for donations. Subsequently, SW said that perhaps it
   would be helpful to discuss how she and the rest of the family are
   coping and the nursing staff will be addressing donation issues. Pt
   mother was open for a discussion.
   Pt
s mother appeared sad and mildly anxious. She said that has gone
   through a lot in the recent past and is not sure how to cope w/her
   daughter
s fatal illness. She also expressed a concern about her
   family. Further, she stated that her daughter has only a few days left.
   SW asked if this was the doctor
[**Initials (NamePattern4) **] [**Last Name (NamePattern4) 1517**]. She replied,
more or
   less
. SW encouraged the pt
s mother to discuss m

### Instruction:
Step-bystep: 
1. Analyse the text, 
2. Select whether the Employment status is explicitly stated, and 
3. Select one of the following labels [employed, unemployed, underemployed, disability, retired, student, unknown].

### Input:
Social Work
   SW following family throughout week for emotional support and resource
   assistance.  Have met with family daily who report to be and appear to
   be coping well given circumstances of pt
s condition and unknown
   prognosis compounded with their distance from their home.  SW arranged
   for family to stay in [**Hospital1 1**] apts in the Galleria.  Family can remain
   there as long as apts are available.  Family and pt seem to have good
   support network, but note they have not yet enlisted the help of others
   and are waiting to do so when they need to return to the west coast.
   Pt
s son returned to CA on [**9-21**], pt
s dtr and husband will return to
   [**Name (NI) 981**], [**Name (NI) 951**] tomorrow.  Pt
s sister [

In [34]:
subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = random.sample(notes, 20)
notes[1]

(64407,
 175035,
 619590,
 datetime.datetime(2169, 12, 29, 9, 43),
 'Chief Complaint:\n   I saw and examined the patient, and was physically present with the ICU\n   Resident for key portions of the services provided.  I agree with his /\n   her note above, including assessment and plan.\n   HPI:\n   59 yo ESRD on HD, dilated CM, admitted w/ CP and dyspnea after several\n   days of missed HD. Probable respiratory arrest from acute CHF, unlikely\n   PEA but presented with K>8. Active issues include fluid overload/ pulm\n   edema, presumed aspiration pna, diarrhea, agitation and presumptive\n   opiate withdrawal.\n   24 Hour Events:\n   Intubated yest for agitation. Psych c/s obtained, CT head done, echo\n   ordered.\n   Allergies:\n   No Known Drug Allergies\n   Last dose of Antibiotics:\n   Vancomycin - [**2169-12-26**] 10:51 AM\n   Piperacillin/Tazobactam (Zosyn) - [**2169-12-28**] 08:29 AM\n   Infusions:\n   Propofol - 80 mcg/Kg/min\n   Other ICU medications:\n   Lorazepam (Ativan) -

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model = "I:/Llama-2-13b-hf"     # o Llama-2-7b(-chat)-hf
adapter_id = "I:/LlamaCare"          # ruta del LoRA

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # si GPU no soporta bfloat16, usa torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_id)
model.eval()




In [ ]:
import utils_llm as ul
from tqdm import tqdm
import pandas as pd
import random

subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = random.sample(notes, 2)
# notes = notes[0:50]

sdoh_output = []

for note in notes:
    meta = {
        'subject_id': note[0],
        'hadm_id': note[1],
        'row_id': note[2],
        'charttime': note[3].isoformat() if note[3] else None
    }

    outputs_per_note = meta.copy()  # Start with metadata

    for sdoh in tqdm(['Employment status', 'Housing issues', 'Transportation issues', 'Parental status', 'Relationship status', 'Social support']):
        instruction = {
            'Employment status': 'Employment status: Whether the patient is currently employed, unemployed, underemployed, disability, retired, student, or unknown. LABELS: [employed, unemployed, underemployed, disability, retired, student, unknown]',
            'Housing issues': 'Housing issues: Any mention of financial status, undomiciled, other. LABELS: [financial status, undomiciled, other, unknown]', 
            'Transportation issues': 'Transportation issues: Any reference to transportation difficulties such as distance, resources, other. LABELS: [distance, resources, other, unknown]', 
            'Parental status':'Parental status: Whether the patient has a child under 18 years old. LABELS: [yes, no, unknown]',
            'Relationship status': 'Relationship status: Whether the patient is widowed, divorced, single. LABELS: [married, partnered, widowed, divorced, single, unknown]',
            'Social support': 'Social support: It does include informal or emotional support from family members, friends, or romantic partners unless such support is clearly mediated through a formal care plan by a social worker or case manager. LABELS: [presence, absence, unknown]'
        }

        prompt = ul.sdh_single_prompt(note[4], sdoh, instruction[sdoh])

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=600,
                length_penalty=1.6,
                num_beams=10,
                no_repeat_ngram_size=3,
                temperature=0.8,
                do_sample=True,
                top_k=15,
                top_p=0.95,
                repetition_penalty=2.1,
                early_stopping=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        outputs_per_note[sdoh] = output_text

    sdoh_output.append(outputs_per_note)


In [ ]:
sdoh_output

In [ ]:
prompt = ul.sdh_single_prompt(note[4], sdoh, instruction[sdoh])
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=600,
        length_penalty=1.6,
        num_beams=10,
        no_repeat_ngram_size=3,
        temperature=1,
        do_sample=True,
        top_k=15,
        top_p=0.95,
        repetition_penalty=2.1,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

#### Strategy prompt # 1

In [ ]:
note_text = note[4]
sdoh = 'Employment status'
sdoh_def = 'whether the patient is currently employed, unemployed, underemployed, disability, retired, student, or unknown. LABELS: [employed, unemployed, underemployed, disability, retired, student, unknown]'
LIST_OF_POSSIBLE_VALUES = ['employed', 'unemployed', 'underemployed', 'disability', 'retired', 'student', 'unknown']
prompt = f"""You are extracting a specific Social Determinants of Health (SDOH) variable from a clinical note.  
Here is the variable definition:  
{sdoh_def}

Step 1. Search the text for information relevant to the SDOH variable as defined above. [Knowledge]  

Step 2. Summarize this information in one or two sentences that are sufficient to determine the value of the SDOH variable. [Summary]  

Step 3. Choose **only** one value for the SDOH variable from the following list: {LIST_OF_POSSIBLE_VALUES}.  
Respond with the exact value only, no explanation or additional text. [Answer]  

Clinical note: \"\"\"{note_text}\"\"\"

"""

In [ ]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(base_model, use_fast=False)
system = "You are a careful medical NLP assistant."
user = f"""You are extracting an SDOH variable.

Definition:
Employment status refers to whether the patient is currently employed, unemployed, retired, a student, or not working due to disability or other reasons.

Step 1. Search the text for information relevant to the SDOH variable. [Knowledge]
Step 2. Summarize this information in one or two sentences. [Summary]
Step 3. Choose only one value from [employed, unemployed, retired, student, not working due to disability, unknown].
Respond with the exact value only. [Answer]

Clinical note: \"\"\"{note_text}\"\"\"
"""
prompt = f"<s>[INST] <<SYS>>{system}<</SYS>>\n\n{user} [/INST]"
prompt = tok(prompt, return_tensors="pt").to(model.device)


In [ ]:
gen = model.generate(
    **prompt,
    max_new_tokens=4,
    do_sample=False,
    temperature=0.0,
    top_p=1.0,
    repetition_penalty=1.1,
    eos_token_id=tok.eos_token_id,
    pad_token_id=tok.eos_token_id,
)
out = tok.decode(gen[0], skip_special_tokens=True)


In [ ]:
out 

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=5000,
        length_penalty=1.6,
        num_beams=10,
        no_repeat_ngram_size=3,
        temperature=0.0,
        do_sample=True,
        top_k=15,
        top_p=0.95,
        repetition_penalty=2.1,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
text = text.replace(prompt, "").strip()
text

In [ ]:
note[4]